In [ ]:
#cord Data(.parquet)를 이미지와 txt파일로 변환하여 저장하는 코드
#https://huggingface.co/datasets/naver-clova-ix/cord-v2


import pandas as pd
import os
from PIL import Image
import io

# Parquet 파일 경로
file_path = "/data/ephemeral/home/MCG/New_Data_Trans/New_Data/cord2/validation-00000-of-00001-cc3c5779fe22e8ca.parquet"

# 데이터 저장 경로 설정
image_save_dir = "/data/ephemeral/home/MCG/New_Data_Trans/cord2_trans_data/images"
text_save_dir = "/data/ephemeral/home/MCG/New_Data_Trans/cord2_trans_data/ground_truths"

# 디렉토리 생성
os.makedirs(image_save_dir, exist_ok=True)
os.makedirs(text_save_dir, exist_ok=True)

# Parquet 파일 로드
df = pd.read_parquet(file_path)

# 데이터 저장
for index, row in df.iterrows():
    try:
        # 이미지와 ground_truth 가져오기
        image_data = row["image"]["bytes"]  # 이미지 바이너리 데이터
        ground_truth = row["ground_truth"]  # OCR 텍스트 데이터

        # 이미지 저장
        image = Image.open(io.BytesIO(image_data))
        image_file_path = os.path.join(image_save_dir, f"cord2_val_image_{index}.jpg")
        image.save(image_file_path)

        # 텍스트 파일 저장
        text_file_path = os.path.join(text_save_dir, f"cord2_val_text_{index}.txt")
        with open(text_file_path, "w") as text_file:
            text_file.write(ground_truth)

        print(f"Saved image {index} and its ground truth.")
        
    except Exception as e:
        print(f"Error processing index {index}: {e}")


In [ ]:
#cord Data의 좌표값을 이미지에 Plot하여 저장하는는 코드드

import os
import json
import cv2
import numpy as np

# 경로 설정
data_folder = "/data/ephemeral/home/MCG/New_Data_Trans/cord1_trans_data"
ground_truth_folder = os.path.join(data_folder, "ground_truths")
image_folder = os.path.join(data_folder, "images")
output_folder = os.path.join(data_folder, "annotated_images")

# 출력 폴더가 없으면 생성
os.makedirs(output_folder, exist_ok=True)

# 모든 ground truth 파일을 반복 처리
for filename in os.listdir(ground_truth_folder):
    if filename.endswith(".txt"):
        txt_path = os.path.join(ground_truth_folder, filename)
        
        # 텍스트 파일 열기
        with open(txt_path, 'r') as file:
            data = json.load(file)

        # 이미지 파일 경로 설정
        image_name = filename.replace("_text_", "_image_").replace(".txt", ".jpg")
        image_path = os.path.join(image_folder, image_name)
        
        # 이미지 읽기
        image = cv2.imread(image_path)
        if image is None:
            print(f"Image not found: {image_path}")
            continue

        # 텍스트 영역 박스 그리기
        for item in data.get("valid_line", []):
            for word in item.get("words", []):
                quad = word.get("quad", {})
                
                # 좌표 유효성 확인
                if all(k in quad for k in ["x1", "y1", "x2", "y2", "x3", "y3", "x4", "y4"]):
                    # 사각형 좌표 가져오기
                    x1, y1 = quad["x1"], quad["y1"]
                    x2, y2 = quad["x2"], quad["y2"]
                    x3, y3 = quad["x3"], quad["y3"]
                    x4, y4 = quad["x4"], quad["y4"]
                    
                    # 좌표값이 숫자인지 확인
                    if all(isinstance(v, (int, float)) for v in [x1, y1, x2, y2, x3, y3, x4, y4]):
                        # 박스 그리기
                        points = [(int(x1), int(y1)), (int(x2), int(y2)), (int(x3), int(y3)), (int(x4), int(y4))]
                        cv2.polylines(image, [np.array(points)], isClosed=True, color=(0, 255, 0), thickness=2)
                        
                        '''# 텍스트 추가
                        text = word.get("text", "")
                        cv2.putText(image, text, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)'''
                    else:
                        print(f"Invalid coordinate values for word '{word.get('text', '')}' in file: {filename}")
                else:
                    print(f"Missing coordinates for word '{word.get('text', '')}' in file: {filename}")

        # dontcare 영역 그리기
        for zone in data.get("dontcare", []):
            for coords in zone:
                # 좌표 유효성 확인
                if all(k in coords for k in ["x1", "y1", "x2", "y2", "x3", "y3", "x4", "y4"]):
                    x1, y1 = coords["x1"], coords["y1"]
                    x2, y2 = coords["x2"], coords["y2"]
                    x3, y3 = coords["x3"], coords["y3"]
                    x4, y4 = coords["x4"], coords["y4"]
                    
                    points = [(int(x1), int(y1)), (int(x2), int(y2)), (int(x3), int(y3)), (int(x4), int(y4))]
                    cv2.polylines(image, [np.array(points)], isClosed=True, color=(255, 0, 0), thickness=2)
                else:
                    print(f"Missing coordinates in dontcare zone in file: {filename}")

        # 점선 부분 그리기 (repeating_symbol)
        for symbol_list in data.get("repeating_symbol", []):
            for symbol in symbol_list:
                quad = symbol.get("quad", {})
                text = symbol.get("text", "")
                
                # 좌표 유효성 확인
                if all(k in quad for k in ["x1", "y1", "x2", "y2", "x3", "y3", "x4", "y4"]):
                    x1, y1 = quad["x1"], quad["y1"]
                    x2, y2 = quad["x2"], quad["y2"]
                    x3, y3 = quad["x3"], quad["y3"]
                    x4, y4 = quad["x4"], quad["y4"]
                    
                    if all(isinstance(v, (int, float)) for v in [x1, y1, x2, y2, x3, y3, x4, y4]):
                        points = [(int(x1), int(y1)), (int(x2), int(y2)), (int(x3), int(y3)), (int(x4), int(y4))]
                        # 점선 그리기
                        cv2.polylines(image, [np.array(points)], isClosed=True, color=(0, 0, 255), thickness=1, lineType=cv2.LINE_AA)
                        
                        # 점선의 경우 텍스트를 표시하지 않음
                    else:
                        print(f"Invalid coordinate values for repeating symbol in file: {filename}")
                else:
                    print(f"Missing coordinates for repeating symbol in file: {filename}")

        # 주석이 추가된 이미지 저장
        output_path = os.path.join(output_folder, image_name)
        cv2.imwrite(output_path, image)
        print(f"Annotated image saved: {output_path}")


Annotated image saved: /data/ephemeral/home/MCG/New_Data_Trans/cord1_trans_data/annotated_images/cord1_train2_image_193.jpg
Annotated image saved: /data/ephemeral/home/MCG/New_Data_Trans/cord1_trans_data/annotated_images/cord1_val_image_2.jpg
Annotated image saved: /data/ephemeral/home/MCG/New_Data_Trans/cord1_trans_data/annotated_images/cord1_train4_image_120.jpg
Annotated image saved: /data/ephemeral/home/MCG/New_Data_Trans/cord1_trans_data/annotated_images/cord1_train3_image_82.jpg
Annotated image saved: /data/ephemeral/home/MCG/New_Data_Trans/cord1_trans_data/annotated_images/cord1_train3_image_166.jpg
Annotated image saved: /data/ephemeral/home/MCG/New_Data_Trans/cord1_trans_data/annotated_images/cord1_train1_image_148.jpg
Annotated image saved: /data/ephemeral/home/MCG/New_Data_Trans/cord1_trans_data/annotated_images/cord1_train3_image_35.jpg
Annotated image saved: /data/ephemeral/home/MCG/New_Data_Trans/cord1_trans_data/annotated_images/cord1_train3_image_144.jpg
Annotated image

In [ ]:
#txt를 UFO 포맷으로 변경하여 자장하는 코드


import os
import json
from datetime import datetime

def organize_annotation_data(input_folder, output_file):
    data = {
        "images": {}
    }
    
    # 모든 텍스트 파일을 읽어서 JSON 데이터로 변환
    for filename in os.listdir(input_folder):
        if filename.endswith(".txt"):
            # 이미지 ID와 JSON 파일 경로 설정
            # 텍스트 파일 이름에서 _text_를 _image_로 대체하여 이미지 파일 이름 생성
            image_id = filename.replace("_text_", "_image_").replace(".txt", ".jpg")
            file_path = os.path.join(input_folder, filename)
            
            # JSON 파일 읽기
            with open(file_path, "r") as f:
                annotation_data = json.load(f)
            
            # 필요한 정보를 추출하여 새로운 데이터 구조로 변환
            words = {}
            # valid_line의 텍스트 박스 처리
            idx = 1
            for line in annotation_data.get("valid_line", []):
                for word in line.get("words", []):
                    quad = word.get("quad", {})
                    if quad:
                        points = [
                            [quad["x1"], quad["y1"]],
                            [quad["x2"], quad["y2"]],
                            [quad["x3"], quad["y3"]],
                            [quad["x4"], quad["y4"]]
                        ]
                        words[f"{idx:04d}"] = {
                            "transcription": word.get("text", ""),
                            "points": points
                        }
                        idx += 1
            
            # repeating_symbol의 점선 또는 반복 기호 처리
            for symbol_list in annotation_data.get("repeating_symbol", []):
                for symbol in symbol_list:
                    quad = symbol.get("quad", {})
                    if quad:
                        points = [
                            [quad["x1"], quad["y1"]],
                            [quad["x2"], quad["y2"]],
                            [quad["x3"], quad["y3"]],
                            [quad["x4"], quad["y4"]]
                        ]
                        words[f"{idx:04d}"] = {
                            "transcription": "",
                            "points": points
                        }
                        idx += 1

            # JSON 구조 생성
            data["images"][image_id] = {
                "paragraphs": {},
                "words": words,
                "chars": {},
                "img_w": annotation_data.get("meta", {}).get("image_size", {}).get("width", 0),
                "img_h": annotation_data.get("meta", {}).get("image_size", {}).get("height", 0),
                "num_patches": None,
                "tags": [],
                "relations": {},
                "annotation_log": {
                    "worker": "worker",
                    "timestamp": datetime.now().strftime("%Y-%m-%d"),
                    "tool_version": "",
                    "source": None
                },
                "license_tag": {
                    "usability": True,
                    "public": False,
                    "commercial": True,
                    "type": None,
                    "holder": "Upstage"
                }
            }
    
    # 정리된 데이터 저장
    with open(output_file, "w") as outfile:
        json.dump(data, outfile, indent=4, ensure_ascii=False)

# 경로 설정
input_folder = "/data/ephemeral/home/MCG/New_Data_Trans/cord2_trans_data/ground_truths"  # 텍스트 파일들이 있는 폴더 경로
output_file = "/data/ephemeral/home/MCG/New_Data_Trans/cord2_trans_data/cord2.json"  # 출력 JSON 파일 경로

# 함수 실행
organize_annotation_data(input_folder, output_file)
